# 공통

In [1]:
import pandas as pd
import pickle
import ast

from tqdm import tqdm
import os
import random
import re
import warnings

os.getcwd()
tqdm.pandas()
warnings.filterwarnings('ignore')

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp, Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

match = pd.read_csv("/data/log-data-2024/Final_20241127/Meta_20241127_optimized.csv")
match = dict(zip(match["optimized_new_contents_id"], match["contents_id"]))

prediction = pd.read_csv("/data/log-data-2024/Final_20241127/8man_sample_20241127_predicted.csv")
prediction["treatment1 prediction"] = prediction["treatment1 prediction"].apply(lambda x : ast.literal_eval(x))

df = pd.read_csv("/data/log-data-2024/2.sequence_generate_ksc/data/sequence_device_match_241127.csv")
cf = pd.read_csv("/data/log-data-2024/Final_20241127/input_search_final_20241127.txt", sep = "\t", header = None)
sequence = pd.concat([df, cf], axis = 1)
sequence = dict(zip(sequence["device_id"], sequence[0]))
prediction["sequence"] = [sequence[i].split()[1:] for i in prediction["treatment1"]]

with open(file='/data/log-data-2024/Final_20241127/match_dict_final2.pickle', mode='rb') as f:
    match_dict = pickle.load(f)
rev = dict(zip(match_dict.values(), match_dict.keys()))

with open(file='/data/log-data-2024/Final_20241127/content_meta_dict_20241127.pickle', mode='rb') as f:
    meta = pickle.load(f)

temp = []
for i in tqdm(prediction["treatment1 prediction"]):
    ttemp = {}
    for w in i:
        try:
            ttemp[int(w)] = meta[int(w)]
        except:
            pass
    temp.append(ttemp)
prediction["treatment1 prediction meta"] = temp

temp = []
for i in tqdm(prediction["sequence"]):
    ttemp = {}
    for w in i:
        try:
            ttemp[int(w)] = meta[int(w)]
        except:
            pass
    temp.append(ttemp)
    
prediction["treatment1 sequence meta"] = temp

sequence = prediction.copy()
sequence

100%|█████████████████████████████████| 20000/20000 [00:00<00:00, 247631.99it/s]


,Unnamed: 0,treatment1,treatment1 prediction,sequence,treatment1 prediction meta,treatment1 sequence meta
0,0,++6Ij698f1UjH/UoA6LQ4uJ9hDeOSaNa4fa7w+uQt9A=,"[181, 8397, 4835, 21757, 23195, 136, 10293, 12...","[8, 8, 8, 131, 181]","{181: 'genre_info: [adventure, comedy, kids, a...","{8: 'genre_info: [adventure, action], actor_in..."
1,1,++6Wwz7+9Ws6Ac+Z5fjt9jKZGK3BbQs9iy3j7zhrcr0=,"[293, 8971, 97, 278, 5640, 878, 17646, 17651, ...","[22, 22, 221, 293]","{293: 'genre_info: [history, drama], actor_inf...","{22: 'genre_info: [horror, kids, fantasy, anim..."
2,2,++DQzvdVvXjmXdd8QTAi+ugAXNX9Yf9uvjZw+b+7Apg=,"[24, 28310, 19385, 20362, 22539, 17858, 7915, ...","[94, 55, 24]","{24: 'genre_info: etc, actor_info: ['Nick Krol...","{94: 'genre_info: etc, actor_info: ['Nick Krol..."
3,3,++EdpY5Ed/MhKap4CmuKWeCanKVsRxyksb0xwlP5Q+Y=,"[17188, 1729, 2923, 1393, 11407, 2340, 1512, 3...","[314, 16910, 16910, 582, 896, 16987, 896, 20972]","{17188: 'genre_info: [drama], actor_info: ['Ja...","{314: 'genre_info: [comedy], actor_info: ['Bra..."
4,4,++PsEaIqZUiphmrWChYq9XCiVkOmoZkRjAli4AFi7pc=,"[72, 19348, 7426, 17501, 21950, 17954, 14167, ...","[31, 72, 72]","{72: 'genre_info: [adventure, action], actor_i...","{31: 'genre_info: [adventure, kids, action], a..."
...,...,...,...,...,...,...
19995,19995,LSxZcTVlW4tN+k/NCQROI9a70YJH8y4GNvcp1wACxz4=,"[1791, 1521, 781, 8395, 104, 2240, 23432, 8382...","[485, 35, 24044, 1791]","{1791: 'genre_info: [drama, romance, melodrama...","{485: 'genre_info: [comedy], actor_info: ['Anj..."
19996,19996,LSzNONtn7O3QAFPjERVgOK/2fB0h9xQ24gL0Jg7Qkak=,"[33, 27913, 11726, 13123, 7155, 26438, 27159, ...","[24045, 56, 33, 33, 33, 33, 33, 33]","{33: 'genre_info: [adventure, comedy, kids, an...","{24045: 'genre_info: [mystery], actor_info: ['..."
19997,19997,LT6AeVu6nFtYQDQ3iDDMsc/p3S1fC0ajS3bsf1fspaE=,"[3584, 848, 17, 642, 2739, 2311, 423, 20, 664,...","[24035, 2, 2, 2, 872, 1163, 3, 2474, 1033, 433...","{3584: 'genre_info: None, actor_info: ['Angeli...","{24035: 'genre_info: etc, actor_info: [], dire..."
19998,19998,LT7m/rrEH0WKljzWwBk5k+SH33aicX1DP9Fv6+mVdBk=,"[10669, 1423, 14360, 13013, 9015, 24, 3810, 24...","[46, 46, 10669]","{10669: 'genre_info: [music], actor_info: ['Th...","{46: 'genre_info: [comedy, kids, fantasy], act..."


In [2]:
z = re.compile("genre_info: \[.+\], actor_info")
genres = []
for i in meta:
    try:
        temp = z.findall(meta[i])[0][13:-13].split(", ")
        for w in temp:
            genres.append(w)
    except:
        pass
genres = list(set(genres))
genre_list = []
for i in genres:
    genre_list.append(i)
    
genre_list.remove("kids")

In [3]:
os.environ["OLLAMA_LLM_LIBRARY"] = "cpu_avx2"

# model_name = 'llama3.1:70b'
model_name = 'llama3.1'

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = Ollama(
    model=model_name,
    temperature=0.75,
    verbose=False
)

In [4]:
from collections import Counter

gl = []
for i in sequence["treatment1 prediction meta"]:
    gen = []
    for w in i:
        try:
            for p in z.findall(i[w])[0][13:-13].split(", "):
                gen.append(p)
        except:
            pass
    length = len(gen)
    gen = dict(sorted(Counter(gen).items(), key=lambda x: x[1], reverse=True))

    genre_list_temp = []
    ratio = 0
    for i in gen:
        if (ratio < 0.4) and (gen[i] > 3):
            ratio += gen[i]/length
            genre_list_temp.append(i)
        else:
            break
    try:
        genre_list_temp.remove("kids")
    except:
        pass
    gl.append(genre_list_temp)
            
sequence["major_genre"] = gl

In [5]:
nt = re.compile("genre_info: None")

result = []

for num in tqdm(sequence.index):

    front_order = []
    back_order = []
    for i in sequence["treatment1 prediction"].loc[num]:
        m = sequence["treatment1 prediction meta"].loc[num]
        gl = sequence["major_genre"].loc[num]
        
        try:
            meta_temp = m[i]

            try:
                meta_temp = z.findall(meta[i])[0][13:-13].split(", ")
            except:
                meta_temp = [nt.findall(meta_temp)[0][12:]]

            if any(element in meta_temp for element in gl):
                front_order.append(i)
            else:
                back_order.append(i)

            front_back = front_order + back_order
        except:
            pass
    result.append(front_back)
    
sequence["treatment1 prediction"] = result

100%|████████████████████████████████████| 20000/20000 [00:22<00:00, 901.16it/s]


In [7]:
temp1 = []
temp2 = []
for i in tqdm(sequence["treatment1 prediction"]):
    ttemp1 = []
    ttemp2 = []
    for w in i:
        try:
            ttemp1.append(match_dict[str(w)])
            ttemp2.append(match[int(w)])
        except:
            pass
    temp1.append(ttemp1)
    temp2.append(ttemp2)
    
sequence["treatment1 prediction name"] = temp1
sequence["treatment1 prediction set_id"] = temp2

temp1 = []
temp2 = []
for i in tqdm(sequence["sequence"]):
    ttemp1 = []
    ttemp2 = []
    for w in i:
        try:
            ttemp1.append(match_dict[str(w)])
            ttemp2.append(match[int(w)])
        except:
            pass
    temp1.append(ttemp1)
    temp2.append(ttemp2)
    
sequence["treatment1 sequence name"] = temp1
sequence["treatment1 sequence set_id"] = temp2

100%|█████████████████████████████████| 20000/20000 [00:00<00:00, 189578.63it/s]


In [20]:
# sequence.to_csv("/data/log-data-2024/20241127_Final/8man_sample_20241127_final.csv")
sequence = pd.read_csv("/data/log-data-2024/20241127_Final/8man_sample_20241127_final.csv")
sequence = sequence[["treatment1", "treatment1 sequence name", "treatment1 prediction name", "treatment1 sequence meta", "major_genre"]]
sequence.columns = ["device_id", "sequence", "prediction", "meta", "major_genre"]

In [29]:
bad_examples = ["Top dramas for you", "We recommend exciting adventure contents!", "Thrill-seekers likes to see our latest picks"]

def generate_message(table):
    res = []
    
    system = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    Your are a smart TV recommendation message generator.
    <|eot_id|>
    '''
    
    user = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    Generate a creative recommendation message to a customer who has previously watched {genre}.
    Do not specify the name of the contents or the name of the genre.
    Instead, define the characteristics of the customer with the given genres.
    The message should be a sentence consists of maximum 8 words.
    Simply output one recommendation message only.
    
    Here's some examples we prefer:
    ["Drama lovers may also like our latest picks.",
    "Try contents that are admired by humor seekers.",
    "You might find yourself on the edge of your seat with this gripping, spine-tingling experience."]
    
    Here's some bad examples we do not prefer:
    {bad_examples}
    <|eot_id|>
    '''
    
    for i in tqdm(table.index):

        template_text = system + user

        prompt = PromptTemplate.from_template(template_text)

        formatted_inputs = {
            "genre" : table["major_genre"].loc[i],
            "bad_examples" : bad_examples
        }

        chain = prompt | llm | StrOutputParser()
        response = chain.invoke(formatted_inputs)
        
        res.append(response)
        
    res = pd.DataFrame({"device_id" : table["device_id"], "message" : res, "sequence" : table["sequence"],
                        "prediction" : table["prediction"], "meta" : table["meta"],
                        "major_genre" : table["major_genre"]})
    
    return res

def approve_message(table):
    
    system = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    You are a marketing manager.
    <|eot_id|>
    '''
    user = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    Evaluate whether the message meets the conditions below.
    Output "Approved" if the message meets all conditions below, otherwise "Rejected".
    Output the basis of the decision with the decision per condition.
    Message: {msg}
    Condition 1. The message should not recommend specific contents or genres.
    Condition 2. Recommending contents to specific type of customers is allowed.
    Condition 3: The message should be grammatically correct and fluent.
    Do not check other conditions than listed.
    Genres are listed in {grs}
    <|eot_id|>
    '''

    template_text = system + user

    prompt = PromptTemplate.from_template(template_text)

    filtering = []
    for i in table.index:
        classify = 1
        target = table["message"].loc[i]
        comp = table["major_genre"].loc[i]
        if ("Thrill" in target) & ("thriller" not in comp):
            classify = 0
        elif ("thrill" in target) & ("thriller" not in comp):
            classify = 0
        else:
            compare = list(set(genre_list) - set(comp))

            for w in compare:
                if w in target:
                    classify = 0
                    break
                else:
                    pass
        if classify == 1:
            filtering.append(1)
        else:
            filtering.append(0)
            
    result = []
    for a, i in tqdm(enumerate(table.index)):
        if filtering[a] == 0:
            result.append(0)
        else:
            formatted_inputs = {
                "msg" : target,
                "grs" : genres,
                "meta": table["meta"].loc[i]
            }

            chain = prompt | llm | StrOutputParser()
            response = chain.invoke(formatted_inputs)

            result.append(response)
            
    table["class"] = result
    table["class"] = [0 if i == 0 else 0 if "Rejected" in i else 0 if (("Rejected" not in i) & ("Approved" not in i)) else 1 for i in table["class"]]
    
    return table

In [38]:
result = pd.DataFrame(columns = ["device_id", "message", "sequence", "prediction", "meta", "class"])
seq = sequence.copy()
df = seq.copy()

while len(result.index) < len(seq.index) * (2/5):
    df = generate_message(df)
    df = approve_message(df)
    
    approved = df[df["class"] == 1]
    df = df[df["class"] == 0]
    
    bad_examples = list(df["message"].loc[:10])
    result = pd.concat([result, approved], axis = 0).reset_index(drop = True)
    
result.to_csv("20241127_sasrec_treatment.csv")
df.to_csv("20241127_sasrec_control.csv")

100%|███████████████████████████████████| 20000/20000 [5:00:02<00:00,  1.11it/s]
20000it [7:04:26,  1.27s/it]
100%|███████████████████████████████████| 16357/16357 [3:37:33<00:00,  1.25it/s]
16357it [4:18:52,  1.05it/s]
100%|███████████████████████████████████| 14524/14524 [3:26:04<00:00,  1.17it/s]
14524it [5:39:23,  1.40s/it]
100%|███████████████████████████████████| 12639/12639 [2:27:52<00:00,  1.42it/s]
12639it [3:43:18,  1.06s/it]


In [37]:
result.to_csv("sample_message.csv")

In [33]:
for i in result["message"]:
    print(i)

"Laughter and whimsy fans may enjoy a lighthearted experience."
"Those who enjoy humor and whimsical storylines might like it."
"People who crave non-stop suspenseful and high-energy storylines."
"Fans of addictive and pulse-pounding plotlines may enjoy this."
"Thrill-seekers often adore shows with complex mysteries."
"Viewers who crave excitement and intense plot twists might enjoy this new release."


In [32]:
result["message"]

0    "Laughter and whimsy fans may enjoy a lighthea...
1    "Those who enjoy humor and whimsical storyline...
2    "People who crave non-stop suspenseful and hig...
3    "Fans of addictive and pulse-pounding plotline...
4    "Thrill-seekers often adore shows with complex...
5    "Viewers who crave excitement and intense plot...
Name: message, dtype: object

In [ ]:
seq = sequence.copy()
df = seq.copy()

df = generate_message(df)
df.to_csv("20241122_sasrec_able.csv")

  0%|                                        | 1/6952 [00:03<6:15:29,  3.24s/it]

In [14]:
# def cleanse(string):
        
#     r = re.compile('\".*?\"')
#     a = re.compile(".+:")
    
#     try:
#         string = r.findall(string)[0]
#     except:
#         pass

#     string = re.sub(a, " ", string)
#     return string.strip()

# def generate_message(table):
#     result = []
    
#     system1 = '''
#     <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
#     You are a marketing manager responsible to generate recommendation messages for smart TV recommendation system.
#     <|eot_id|>
#     '''

#     user1 = '''
#     <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
#     I will provide you "a squence of contents" that a specific customer has previously viewed.
#     Summarize the smart TV contents consumption patterns of the customer in one sentence.
#     Concentrate on the common points of the contents if possible.
#     Simply output the summary only.
#     You should retrieve the "information of the contents" to identify which contents are assigned to each number.
#     A squence of contents: {sequence}
#     Information of the contents: {meta}.
#     <|eot_id|>
#     '''

#     system2 = '''
#     <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
#     You are a marketing manager responsible to generate recommendation messages for smart TV recommendation system.
#     <|eot_id|>
#     '''

#     user2 = '''
#     <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
#     Based on the following "characteristics of a customer", generate a recommendation message that encourage customers to click-through the recommended contents.
#     The recommendation message should be a sentence consists of 15 - 60 characters.
#     The recommended contents are already determined: you should not recommend specific contents or genres.
#     Focus only on the characteristics of the customer, instead of focusing on the recommended contents.
#     You should use only general nouns when referring to recommended contents, such as contents, programs, recommendations, and picks.
#     Any modifier for the recommended contents is strictly prohibited. For example, recommending "triller contents" or "comedy programs" is not allowed.
#     Characteristics of the customers: {character}
#     Simply output one recommendation message only.
#     <|eot_id|>
#     '''

#     for i in tqdm(table.index):

#         template_text1 = system1 + user1

#         prompt1 = PromptTemplate.from_template(template_text1)

#         formatted_inputs1 = {
#             "sequence": table["prediction"].loc[i],
#             "meta": table["meta"].loc[i]
#         }

#         chain1 = prompt1 | llm | StrOutputParser()
#         response1 = chain1.invoke(formatted_inputs1)

#         # ------------------------------------------------------------------------

#         template_text2 = system2 + user2

#         prompt2 = PromptTemplate.from_template(template_text2)

#         formatted_inputs2 = {
#             "character": response1
#         }

#         chain2 = prompt2 | llm | StrOutputParser()
#         response2 = chain2.invoke(formatted_inputs2)

#         result.append(response2)
        
#     df = pd.DataFrame({"device_id" : table["device_id"], "message" : result, "sequence" : table["sequence"],
#                        "prediction" : table["prediction"], "meta" : table["meta"], "major_genre" : table["major_genre"]})
    
#     df["message"] = df["message"].apply(lambda x : cleanse(x))
#     return df
    
# def approve_message(table):
    
#     system = '''
#     <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
#     You are a classifier.
#     <|eot_id|>
#     '''
#     user = '''
#     <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
#     I will provide you a recommendation message. Evaluate whether the message meets the conditions below.
#     Output "Approved" if the message meets all conditions below, otherwise "Rejected".
#     Output the basis of the decision with the decision per condition.
#     Message: {msg}
#     Condition 1. The message should not recommend specific contents or genres.
#     Condition 2. Recommending contents to specific type of customers is allowed.
#     Condition 3: The message should be grammatically correct and fluent.
#     Condition 4: The message should be a sentence consists of 15 - 60 characters.
#     Condition 5. Message recommending family-friendly contents is prohibited if the {meta} is "R19".
#     Do not check other conditions than listed.
#     Genres are listed in {grs}
#     <|eot_id|>
#     '''

#     template_text = system + user

#     prompt = PromptTemplate.from_template(template_text)

#     filtering = []
#     for i in table.index:
#         classify = 1
#         target = table["message"].loc[i]
#         comp = table["major_genre"].loc[i]
#         if ("Thrill" in target) & ("thriller" not in comp):
#             classify = 0
#         elif ("thrill" in target) & ("thriller" not in comp):
#             classify = 0
#         else:
#             compare = list(set(genre_list) - set(comp))

#             for w in compare:
#                 if w in target:
#                     classify = 0
#                     break
#                 else:
#                     pass
#         if classify == 1:
#             filtering.append(1)
#         else:
#             filtering.append(0)
            
#     result = []
#     for a, i in tqdm(enumerate(table.index)):
#         if filtering[a] == 0:
#             result.append(0)
#         else:
#             formatted_inputs = {
#                 "msg" : target,
#                 "grs" : genres,
#                 "meta": table["meta"].loc[i]
#             }

#             chain = prompt | llm | StrOutputParser()
#             response = chain.invoke(formatted_inputs)

#             result.append(response)
            
#     table["class"] = result
#     table["class"] = [0 if i == 0 else 0 if "Rejected" in i else 0 if (("Rejected" not in i) & ("Approved" not in i)) else 1 for i in table["class"]]
    
#     return table

In [16]:
result = pd.DataFrame(columns = ["device_id", "message", "sequence", "prediction", "meta", "class"])
seq = sequence.copy()
df = seq.copy()

while len(result.index) != len(seq.index):
    df = generate_message(df)
    df = approve_message(df)
    
    approved = df[df["class"] == 1]
    df = df[df["class"] == 0]
    
    bad_examples = list(df["message"].iloc[:10])
    
    result = pd.concat([result, approved], axis = 0).reset_index(drop = True)
    
    if len(result.index) > len(sequence.index)/3:
        result.to_csv("result_20241118_sasrec_able.csv")
        break
    
result

100%|█████████████████████████████████████| 6952/6952 [8:33:17<00:00,  4.43s/it]
6952it [8:52:59,  4.60s/it]
 84%|█████████████████████████████▍     | 5004/5960 [5:55:05<1:07:50,  4.26s/it]


KeyboardInterrupt: 

In [2]:
df = pd.read_csv("result_20241110.csv")
df

,Unnamed: 0,device_id,message,sequence,prediction,meta,class
0,0,+2Te5oVYRVXD8thHy8BYXJpzQynf+lDb5SS7PHmYTv4=,Imaginative young explorers discover more exc...,"['4', '4', '215', '1544']","[2390, 137, 369, 7362, 3507, 550, 4226, 766, 1...","{1152: 'title: COLLEGE FOOTBALL TODAY, genre_i...",1
1,1,+47TETL0Uf5bTxA4QngSJd7yzvZ3yRp2yWMJCSl3TX8=,Thrill seekers get ready for your next adventure,"['583', '66', '66']","[1026, 2082, 5481, 19, 6195, 61, 704, 174, 165...","{1026: ""title: BEAN, genre_info: [comedy], act...",1
2,2,+4Q50w5ruY4feAgWw7sl2Ei2NPBu/zuvAWmYUt+HP2s=,Discovery seekers and young explorers get rea...,"['442', '11', '224', '224']","[4456, 1645, 196, 5768, 884, 15591, 685, 3098,...","{7: 'title: PAW PATROL, genre_info: [kids, adv...",1
3,3,+5/WvO0usXHW2vI63brjQA82NjjiQihQlJib5gDxJOg=,Thrill seekers and mystery enthusiasts we've ...,"['97', '1221', '27', '27', '27', '27']","[1, 5, 2235, 1758, 1202, 2103, 7, 14, 9617, 26...","{1: 'title: PEACOCK, genre_info: [comedy], act...",1
4,4,+5sUW1ee3E+hqkVYl/AEW46B7iWndCWg5H0nggmEsv0=,Get ready for delightful moments with our cura...,"['618', '618', '618', '110', '110', '110', '61...","[623, 205, 151, 1593, 2684, 528, 257, 996, 109...","{384: ""title: THE DAILY SHOW WITH JON STEWART,...",1
...,...,...,...,...,...,...,...
6947,6947,+CssJpavaipmaCJaAE4k901a2vc//gg0QSnn+/1SzQU=,Get ready for more laughter with our fun picks,"['1409', '325', '4634', '797', '1059', '667', ...","[1648, 403, 3984, 2684, 2867, 151, 215, 116, 2...","{384: ""title: THE DAILY SHOW WITH JON STEWART,...",1
6948,6948,+FdJ0X3pSuPi3jwXwI2XnpBjHfM+K/aiOSt1NyXvAd4=,Viewers who enjoy gripping storylines and matu...,"['6312', '7846', '9153', '7846', '42', '4', '4...","[1043, 12488, 1333, 15869, 4209, 794, 397, 204...","{2048: 'title: EMERGENCY, genre_info: [thrille...",1
6949,6949,+HJxsWz9qYoZxvKdgwznvutCujqb2pLHXKN7/Si8UTM=,Discover more engaging stories like THE VAMPIR...,"['313', '313', '313', '313']","[3070, 673, 1028, 335, 8, 340, 103, 1406, 1340...","{2: 'title: DRAMA, genre_info: [drama], actor_...",1
6950,6950,+KGJUs06gu9sQhIS6TGgzORnQEyPiK9HLjqgOGTlWCY=,Story lovers like you will enjoy our engaging ...,"['5142', '5142', '5142', '5142', '5142']","[868, 84, 847, 16910, 77, 254, 2727, 28, 303, ...","{1: 'title: PEACOCK, genre_info: [comedy], act...",1
